<a href="https://colab.research.google.com/github/shruti629/Deep-Health-Analysis-and-Disease-Prediction/blob/main/Disease_Based_upon_symptoms_%26_Precautions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score ,f1_score
import warnings
warnings.filterwarnings('ignore')
import pickle

In [ ]:
df = df.rename(columns={'MisspelledColumnName': 'Symptoms'})
df=pd.read_csv('/content/drive/MyDrive/disease precaution.csv')

In [ ]:
df.head()

,Disease,Symptoms,Precautions
0,Influenza (Flu),"Fever, cough, sore throat, runny or stuffy nos...","Get vaccinated, wash hands frequently, avoid c..."
1,COVID-19,"Fever, cough, difficulty breathing, loss of ta...","Get vaccinated, wear a mask, practice social d..."
2,Diabetes,"Increased thirst, frequent urination, extreme ...","Maintain a healthy diet, exercise regularly, m..."
3,Hypertension (High Blood Pressure),"Often asymptomatic, but may include headaches,...","Eat a balanced diet low in salt, exercise regu..."
4,Asthma,"Shortness of breath, chest tightness, wheezing...","Avoid allergens, use prescribed inhalers, avoi..."


In [ ]:
# Encode categorical variables
le_disease = LabelEncoder()
df['Disease'] = le_disease.fit_transform(df['Disease'])

In [ ]:
mlb_symptoms = MultiLabelBinarizer()
mlb_precautions = MultiLabelBinarizer()

In [ ]:
df['Symptoms'] = df['Symptoms'].apply(lambda x: x.split(','))

df['Precautions'] = df['Precautions'].apply(lambda x: x.split(','))


In [ ]:
symptoms_encoded = mlb_symptoms.fit_transform(df['Symptoms'])
precautions_encoded = mlb_precautions.fit_transform(df['Precautions'])

In [ ]:
# Split the data into features and target
X = df[['Disease']]
y_symptoms = pd.DataFrame(symptoms_encoded, columns=mlb_symptoms.classes_)
y_precautions = pd.DataFrame(precautions_encoded, columns=mlb_precautions.classes_)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train_symptoms, y_test_symptoms, y_train_precautions, y_test_precautions = train_test_split(
    X, y_symptoms, y_precautions, test_size=0.2, random_state=42)


In [ ]:
# Model training for symptoms
model_symptoms = RandomForestClassifier()
model_symptoms.fit(X_train, y_train_symptoms)

# Model training for precautions
model_precautions = RandomForestClassifier()
model_precautions.fit(X_train, y_train_precautions)


RandomForestClassifier()

In [ ]:
# Predictions
y_pred_symptoms = model_symptoms.predict(X_test)
y_pred_precautions = model_precautions.predict(X_test)


In [ ]:
# Evaluation
precision_symptoms = precision_score(y_test_symptoms, y_pred_symptoms, average='macro', zero_division=1)
recall_symptoms = recall_score(y_test_symptoms, y_pred_symptoms, average='macro', zero_division=1)
f1_symptoms = f1_score(y_test_symptoms, y_pred_symptoms, average='macro', zero_division=1)

precision_precautions = precision_score(y_test_precautions, y_pred_precautions, average='macro', zero_division=1)
recall_precautions = recall_score(y_test_precautions, y_pred_precautions, average='macro', zero_division=1)
f1_precautions = f1_score(y_test_precautions, y_pred_precautions, average='macro', zero_division=1)

print(f'Symptoms - Precision: {precision_symptoms}, Recall: {recall_symptoms}, F1-score: {f1_symptoms}')
print(f'Precautions - Precision: {precision_precautions}, Recall: {recall_precautions}, F1-score: {f1_precautions}')

Symptoms - Precision: 0.7907237152286658, Recall: 0.7392739273927392, F1-score: 0.5639006757818639
Precautions - Precision: 0.7836879432624113, Recall: 0.7695035460992907, F1-score: 0.5921985815602837


In [ ]:
# Cross-validation
cv_scores_symptoms = cross_val_score(model_symptoms, X, symptoms_encoded, cv=5, scoring='f1_macro')
cv_scores_precautions = cross_val_score(model_precautions, X, precautions_encoded, cv=5, scoring='f1_macro')

print(f'Cross-validated F1-scores for symptoms prediction: {cv_scores_symptoms}')
print(f'Cross-validated F1-scores for precautions prediction: {cv_scores_precautions}')

Cross-validated F1-scores for symptoms prediction: [0.0111504  0.00733573 0.00966525 0.01922442 0.0049505 ]
Cross-validated F1-scores for precautions prediction: [0.01384667 0.00413712 0.01040189 0.00641675 0.        ]


In [ ]:
# Function to make predictions
def predict_symptoms_precautions(disease_name):
    try:
        disease_encoded = le_disease.transform([disease_name])
        symptoms_encoded = model_symptoms.predict([disease_encoded])
        precautions_encoded = model_precautions.predict([disease_encoded])

        symptoms = mlb_symptoms.inverse_transform(symptoms_encoded)
        precautions = mlb_precautions.inverse_transform(precautions_encoded)

        return symptoms[0], precautions[0]
    except ValueError as e:
        return str(e), str(e)

In [ ]:
# Example prediction
example_disease = 'Diabetes'
predicted_symptoms, predicted_precautions = predict_symptoms_precautions(example_disease)
print(f'Predicted symptoms for {example_disease}: {predicted_symptoms}')
print(f'Predicted precautions for {example_disease}: {predicted_precautions}')

Predicted symptoms for Diabetes: (' blurry vision', ' extreme fatigue', ' frequent urination', ' slow-healing sores', 'Increased thirst')
Predicted precautions for Diabetes: (' exercise regularly', ' monitor blood sugar levels', ' take prescribed medications', 'Maintain a healthy diet')


Save the models using pickle

In [ ]:
with open('model_symptoms.pkl', 'wb') as f:
    pickle.dump(model_symptoms, f)

with open('model_precautions.pkl', 'wb') as f:
    pickle.dump(model_precautions, f)

with open('le_disease.pkl', 'wb') as f:
    pickle.dump(le_disease, f)

with open('mlb_symptoms.pkl', 'wb') as f:
    pickle.dump(mlb_symptoms, f)

with open('mlb_precautions.pkl', 'wb') as f:
    pickle.dump(mlb_precautions, f)

In [ ]:
# Load the models using pickle
def load_models():
    with open('model_symptoms.pkl', 'rb') as f:
        model_symptoms = pickle.load(f)
    with open('model_precautions.pkl', 'rb') as f:
        model_precautions = pickle.load(f)
    with open('le_disease.pkl', 'rb') as f:
        le_disease = pickle.load(f)
    with open('mlb_symptoms.pkl', 'rb') as f:
        mlb_symptoms = pickle.load(f)
    with open('mlb_precautions.pkl', 'rb') as f:
        mlb_precautions = pickle.load(f)
    return model_symptoms, model_precautions, le_disease, mlb_symptoms, mlb_precautions
